In [57]:
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder

## Data Structure 📖
data[i] -> ith data  
data = label (data[i][0]) + 3d key points (data[i][1])   
3d key point's shape is (195,200) => 195 frames, 200 joint data / frame  
200 joint data => 50 joint x 4 values (x,y,z,confidence)  

In [74]:
data = np.load('dataset_padding_new.npy', allow_pickle=True)

## Data Processing for ST-GCN 🔬

In [ ]:
# this function process raw open pose data to data required by st-gcn
# however, the graph is not built in this step
def process_data(data):
    # convert openpose raw data to st-gcn preferred data
    labels = []
    all_key_points = []
    for label, key_points in data:
        labels.append(label)
        key_points = key_points.reshape(195, 50, 4) # frame-joint-channels(x,y,z,c)
        key_points = key_points.transpose(2, 0, 1) # st-gcn requires (channels, time_stamp, joints)
        all_key_points.append(key_points)
    all_key_points = np.stack(all_key_points)

    # label encoding
    label_encoder = LabelEncoder()
    encoded_label = label_encoder.fit_transform(labels)

    # convert to pytorch-like data
    x = torch.tensor(all_key_points, dtype=torch.float32)
    y = torch.tensor(encoded_label,dtype=torch.long)

    class Dataset:
        def __init__(self, x, y):
            self.X = x
            self.Y = y
        
        def get_length(self):
            return len(self.X)
        
        def get_data(self, idx):
            key_points = self.X[idx]
            label = self.Y[idx]
            return (key_points,label)
    
    return Dataset(x,y), label_encoder